Install dependencies:

In [ ]:
import kagglehub
#from kagglehub import KaggleDatasetAdapter
#import zipfile
import pandas as pd

In [ ]:
import os
os.system("pip3 show kagglehub")

256

Download data from Kaggle:

In [ ]:
path=kagglehub.dataset_download("prestonrobertson7/league-of-legends-data-9292022")


Read into a Pandas DataFrame:

In [ ]:
fileN = "Sep-09-2022_10000matches.csv"
raw =pd.read_csv(path+"/"+fileN)

/tmp/ipykernel_21560/1373787418.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  raw =pd.read_csv(path+"/"+fileN)


**Cleaning the data**
1. Select correct columns
2. Remove ARAMs (this is a random game mode)

In [25]:
pd.DataFrame.head(raw)

,Unnamed: 0,assists,GameID,GameMode,baronKills,bountyLevel,champExperience,champLevel,championName,damageDealtToObjectives,...,totalTimeSpentDead,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0,20,NA1_4430908515,ARAM,0,0,14531,15,Ashe,112,...,74,797,465,1,4,5,0,0,0,False
1,0,3,4035079604,CLASSIC,0,3,15453,16,Sett,18147,...,156,2556,686,4,4,10,0,0,7,True
2,0,19,4035079604,CLASSIC,0,3,13168,15,Trundle,27626,...,82,956,1571,6,4,21,3,0,11,True
3,0,6,4035079604,CLASSIC,0,1,13619,15,Sylas,10629,...,127,600,935,6,4,8,0,0,5,True
4,0,7,4035079604,CLASSIC,1,1,10674,13,KogMaw,13624,...,118,2202,521,3,4,3,1,1,2,True
